In [ ]:
import pandas as pd
import argparse
import os
import openai
import numpy as np
from tqdm import trange
import scipy.special as sp
import re
import time
import random

def extractNum(string):
    match = re.search(r'\b([0-9]|[1-6][0-9]|7[0-6])\b', string)
    try:
        num = int(match.group())
        return num
    except:
        return None
    
openai.api_key = '<your openai token>'

banking77 = pd.read_parquet('banking77.parquet')

sample_ls = list(banking77['text'])

label_ls = [
    "activate_my_card",
    "age_limit",
    "apple_pay_or_google_pay",
    "atm_support",
    "automatic_top_up",
    "balance_not_updated_after_bank_transfer",
    "balance_not_updated_after_cheque_or_cash_deposit",
    "beneficiary_not_allowed",
    "cancel_transfer",
    "card_about_to_expire",
    "card_acceptance",
    "card_arrival",
    "card_delivery_estimate",
    "card_linking",
    "card_not_working",
    "card_payment_fee_charged",
    "card_payment_not_recognised",
    "card_payment_wrong_exchange_rate",
    "card_swallowed",
    "cash_withdrawal_charge",
    "cash_withdrawal_not_recognised",
    "change_pin",
    "compromised_card",
    "contactless_not_working",
    "country_support",
    "declined_card_payment",
    "declined_cash_withdrawal",
    "declined_transfer",
    "direct_debit_payment_not_recognised",
    "disposable_card_limits",
    "edit_personal_details",
    "exchange_charge",
    "exchange_rate",
    "exchange_via_app",
    "extra_charge_on_statement",
    "failed_transfer",
    "fiat_currency_support",
    "get_disposable_virtual_card",
    "get_physical_card",
    "getting_spare_card",
    "getting_virtual_card",
    "lost_or_stolen_card",
    "lost_or_stolen_phone",
    "order_physical_card",
    "passcode_forgotten",
    "pending_card_payment",
    "pending_cash_withdrawal",
    "pending_top_up",
    "pending_transfer",
    "pin_blocked",
    "receiving_money",
    "Refund_not_showing_up",
    "request_refund",
    "reverted_card_payment?",
    "supported_cards_and_currencies",
    "terminate_account",
    "top_up_by_bank_transfer_charge",
    "top_up_by_card_charge",
    "top_up_by_cash_or_cheque",
    "top_up_failed",
    "top_up_limits",
    "top_up_reverted",
    "topping_up_by_card",
    "transaction_charged_twice",
    "transfer_fee_charged",
    "transfer_into_account",
    "transfer_not_received_by_recipient",
    "transfer_timing",
    "unable_to_verify_identity",
    "verify_my_identity",
    "verify_source_of_funds",
    "verify_top_up",
    "virtual_card_not_working",
    "visa_or_mastercard",
    "why_verify_identity",
    "wrong_amount_of_cash_received",
    "wrong_exchange_rate_for_cash_withdrawal",
]

instance_counter = 0
pred_id_ls = []

random.shuffle(sample_ls)
for i_ in trange(10):
    random.shuffle(label_ls)
    prompt_1 = ''
    for label_id_ in range(len(label_ls)):
        prompt_1 = prompt_1 + 'Label ' + str(label_id_) + ': ' + label_ls[label_id_].replace('\n', ' ').replace(':', ' ').replace('_', ' ') + '\n'
    prompt_1 = prompt_1 + 'Target Text: ' + sample_ls[i_] + '\n'
    prompt_1 = prompt_1 + 'Which Label matches the intent of the Target Text best? Answer only one Label index number.'
    
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages = [{"role": "user", "content": prompt_1}],
      temperature = 0.0,
      max_tokens = 100,
    )

    chatgpt_pred = response["choices"][0]['message']['content']

    pred_id_ = extractNum(chatgpt_pred)

    pred_id_ls.append(pred_id_)